In [2]:
'''
dev_id = r.json()["dev_id"]
v1 = r.json()["v1"]
create_ts = r.json()["create_ts"]
port = r.json()["port"]
value = r.json()["value"]
x = [v1,dev_id,port,value,create_ts]
df11= pd.DataFrame(x).transpose()
'''

'\ndev_id = r.json()["dev_id"]\nv1 = r.json()["v1"]\ncreate_ts = r.json()["create_ts"]\nport = r.json()["port"]\nvalue = r.json()["value"]\nx = [v1,dev_id,port,value,create_ts]\ndf11= pd.DataFrame(x).transpose()\n'

In [3]:
import pandas as pd 
import numpy as np

In [4]:
x=[1603626051, "EC4N0003","A1",0.11,"2020-10-21 13:00:02"]
df11= pd.DataFrame(x).transpose()
df11

,0,1,2,3,4
0,1603626051,EC4N0003,A1,0.11,2020-10-21 13:00:02


In [5]:
df11.rename(columns={0: "v1", 1: "dev_id", 2: "port",3: "value", 4: "create_ts"},inplace=True)
df11

,v1,dev_id,port,value,create_ts
0,1603626051,EC4N0003,A1,0.11,2020-10-21 13:00:02


In [6]:
df = pd.read_csv("channel_values.csv")

In [7]:
from clean_data import *
df6 = clean_data(df,"EC4N0003","A0")

In [8]:
df6.dtypes

v1                    int32
dev_id               object
port                 object
value               float64
create_ts    datetime64[ns]
dtype: object

In [9]:
from rolling import *
df2 = rolling_avg1(df6)

In [10]:
df2 = df2.append(df11,ignore_index=True)

In [12]:
df2.tail()

,v1,dev_id,port,value,create_ts,rolling_avg,pred
801257,1603626048,EC4N0003,A0,0.1,2020-10-25 11:40:49,0.1,Idle
801258,1603626048,EC4N0003,A0,0.1,2020-10-25 11:40:49,0.1,Idle
801259,1603626049,EC4N0003,A0,0.1,2020-10-25 11:40:50,0.1,Idle
801260,1603626050,EC4N0003,A0,0.1,2020-10-25 11:40:51,0.1,Idle
801261,1603626051,EC4N0003,A1,0.11,2020-10-21 13:00:02,NaN,NaN


In [57]:
n = 3 #no of signatures

rolling1 = [5,0,4] # rolling average of last 5 observations must be between 0 and 4
max1 = [2,0,5]     # max value of last 2 readings between 0 and 5
min1 = [3,0,4]     # min value of last 3 readings between 0 and 4
std1 = [5,0,1]  # stddev of last 5 readings between 0 and 1




In [13]:
'''
Function calling 

from rollinng import *
df2 = rollinng(df2,4)

from maximum import *
max_values = []
max_times = []
a,b = maximum(df2,4000)
max_values.append(a)
max_times.append(b)

from mean import *
avg = mean(df2,n)

from hmean import hmean
n = 10
hmean = hmean(df2,n)

from stddev import *
std = stddev(df2,n)
std
'''

In [14]:
from rollinng import *
from maximum import *
from minimum import *
from mean import *
from stddev import *

In [24]:
df2 = rollinng(df2,rolling1[-1])
df2.loc[df2.index[-1],'rolling_avg']

0.10200000000000001

In [28]:
df2.value = df2["value"].astype('float')

In [78]:
if((df2.loc[df2.index[-1],'rolling_avg']>rolling1[1]) & (df2.loc[df2.index[-1],'rolling_avg']<rolling1[2])):
    a,b = maximum(df2,max1[0])
    if((a>max1[1])& (a<max1[2])):
        c,d = minimum(df2,min1[0])
        if((c>min1[1]) & (c<min1[2])):
            standev = stddev(df2,std1[0])
            if((standev>std1[1])&(standev<std1[2])):
                    df2.loc[df2.index[-1],'pred']= "Signature1"

In [79]:
df2.tail()

,v1,dev_id,port,value,create_ts,rolling_avg,pred
801257,1603626048,EC4N0003,A0,0.10,2020-10-25 11:40:49,0.100,Idle
801258,1603626048,EC4N0003,A0,0.10,2020-10-25 11:40:49,0.100,Idle
801259,1603626049,EC4N0003,A0,0.10,2020-10-25 11:40:50,0.100,Idle
801260,1603626050,EC4N0003,A0,0.10,2020-10-25 11:40:51,0.100,Idle
801261,1603626051,EC4N0003,A1,0.11,2020-10-21 13:00:02,0.102,Signature1


# ALL FUNCTION DEFINITIONS

In [64]:
def rollinng(df,x):
    import pandas as pd
    import numpy as np
    sum=0
    x=int(x)
    for i in range(0,x):
        sum = sum + float(df.iloc[-1-i,3])
    df.loc[df.index[-1],'rolling_avg'] = sum/x
    return df



In [65]:
def stddev(df1,n):
    df1 = df1[-n:]
    import pandas as pd
    import numpy as np
    std = np.std(df1["value"])
    return std

In [67]:
def maximum(df,n):
    import pandas as pd
    df = df[-n:]
    max_val = df.value.max()
    a = df['value'].idxmax()
    max_time = df['create_ts'][a]
    return max_val,max_time

In [68]:
def minimum(df,n):
    df = df[-n:]
    min_val = df.value.min()
    x = df['value'].idxmin() 
    min_time = df["create_ts"][x]
    return min_val,min_time


In [69]:
def mean(df1,n):
    df1 = df1[-n:]
    avg = df1.value.mean()
    return avg

In [70]:
def hmean(df1,n):
    df1 = df1[-n:]
    import scipy
    from scipy import stats
    return scipy.stats.hmean(df1["value"])
    

In [80]:
def deriv1(df1):
    import pandas as pd
    import numpy as np
    df_avg = df1.groupby('create_ts',sort=False,as_index=False).mean()
    df_avg['diff'] = df_avg['create_ts'] - df_avg['create_ts'].shift(1)
    df_avg["diff"] = df_avg["diff"].dt.total_seconds()
    df_avg["diff"].fillna(0,inplace=True)
    df_avg["diff"] = df_avg["diff"].astype(int)
    df_avg['delta'] = df_avg['value'] - df_avg['value'].shift(1)
    df_avg['derivative'] = df_avg["delta"]/df_avg["diff"]
    return df_avg

In [91]:
# Rolling average for entire dataframe
def rolling_avg1(df):
    import numpy as np
    import pandas as pd
    df['rolling_avg'] = df['value'].rolling(window=4).mean()
    return df

In [92]:
# Predict function for prev data
def predict(df,x):    
    import pandas as pd
    import numpy as np
    df.loc[df.index[-1],'pred'] = np.where(df.loc[df.index[-1],'rolling_avg']>=x, 'Running', 'Idle')
    return df
